In [1]:
import pandas as pd

In [40]:
!pwd

/piech/u/yunsung/analytic-grading/analytic_grading/data/raw


In [24]:
test_df_txt = pd.read_csv('public_leaderboard_rel_2.tsv', delimiter='\t')
test_sol_df = pd.read_csv('public_leaderboard_solution.csv')
train_df = pd.read_csv('train_rel_2.tsv', delimiter='\t')

In [26]:
test_df_txt.head()

,Id,EssaySet,EssayText
0,1673,1,The procedures I think they should have includ...
1,1674,1,"In order to replicate this experiment, you wou..."
2,1675,1,"In order to replicate their experiment, you wo..."
3,1676,1,Pleace a simple of one material into one conta...
4,1677,1,Determin the mass of four different samples ma...


In [27]:
test_sol_df.head()

,id,essay_set,essay_weight,essay_score,Usage
0,1673,1,1,1,PublicTest
1,1674,1,1,1,PublicTest
2,1675,1,1,3,PublicTest
3,1676,1,1,0,PublicTest
4,1677,1,1,0,PublicTest


In [46]:
# merge test_df and test_sol_df based on the 'id' column
test_df = pd.merge(test_df_txt, test_sol_df, left_on='Id', right_on='id', how='left')
test_df = test_df[['Id', 'EssaySet', 'EssayText', 'essay_score']].rename(columns={'essay_score': 'Score1'})

In [49]:
dfs = {}
for i in range(1,11):
    # concatenate train and test df with EssaySet == i
    # create a split column to distinguish between train and test data
    dfs[i] = pd.concat([train_df[train_df['EssaySet'] == i].assign(split='train'), 
                        test_df[test_df['EssaySet'] == i].assign(split='test')])


In [50]:
dfs[1]

,Id,EssaySet,Score1,Score2,EssayText,split
0,1,1,1,1.0,Some additional information that we would need...,train
1,2,1,1,1.0,"After reading the expirement, I realized that ...",train
2,3,1,1,1.0,"What you need is more trials, a control set up...",train
3,4,1,0,0.0,The student should list what rock is better an...,train
4,5,1,2,2.0,For the students to be able to make a replicat...,train
...,...,...,...,...,...,...
552,2225,1,2,NaN,In order some to replicate I need to know how ...,test
553,2226,1,0,NaN,If I wanted to make the same kind of experimen...,test
554,2227,1,1,NaN,In orderto replicate the experiment i would n...,test
555,2228,1,1,NaN,"In order to replicate this experiment, the pro...",test


In [65]:
import os
for i in range(1,11):
    os.makedirs(f'../asap_{i:02d}', exist_ok=True)
    dfs[i].to_csv(f'../asap_{i:02d}/df.csv', index=False)
    # copy entire directory *.pkl from 
    # /piech/u/yunsung/auto-grammar/canongrade/data/processed/asap_{i:02d}/splits/cv*.pkl
    # to ../asap_{i:02d}/splits/

    import glob
    import shutil
    import pickle
    os.makedirs(f'../asap_{i:02d}/splits', exist_ok=True)
    for fname in glob.glob(f'/piech/u/yunsung/auto-grammar/canongrade/data/processed/asap_{i:02d}/splits/cv*.pkl'):
        with open(fname, 'rb') as f:
            split = pickle.load(f)
        
        split['test'] = dfs[i][dfs[i]['split'] == 'test'].Id.values
        with open(f'../asap_{i:02d}/splits/{os.path.basename(fname)}', 'wb') as f:
            pickle.dump(split, f)
    
        

In [68]:
print(len(train_df))
print(len(test_df))

17043
5224


In [69]:
with open('../asap_01/splits/cv1.pkl', 'rb') as f:
    split = pickle.load(f)
print(len(split['train']))
print(len(split['valid']))
print(len(split['test']))

1341
331
557
